In [ ]:
import pandas as pd
import numpy as np
from kmodes.kmodes import KModes
import matplotlib.pyplot as plt

In [ ]:
phq= pd.read_csv('DATOS/phq_num.csv',sep=';')

In [ ]:
phq.head()
phq.shape
phq.isnull().sum()

phq=phq.dropna()

In [ ]:
### Seleccionar las variables
phq_ft=phq.drop(['phq_global','phq_s10','phq_cat'],axis=1)
### Pasar a categoricas
def reco_s00(val):
    if val == 0:
        return 'Nunca'
    elif val==1:
        return 'Algunos'
    elif val ==2:
        return 'Varios'
    elif val ==3:
        return 'Todos'
lista_rec=[phq_ft[i].apply(reco_s00) for i in phq_ft.columns]

phq_rec=pd.DataFrame(lista_rec).transpose()

# Visualización de los datos

In [ ]:
lista=[round(100*phq_rec.value_counts(i)/phq_rec.shape[0],2) for i in phq_rec.columns ]

In [ ]:
phq_gf=(pd.DataFrame(lista,index= phq_rec.columns).
 reset_index().
 rename(columns={'index':'sintoma'}).
 melt(id_vars='sintoma', 
      value_vars=['Nunca','Algunos','Todos','Varios']))


In [ ]:
phq_gf.head()

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=[
    go.Bar(name='Nunca', 
           x=phq_gf.query("variable=='Nunca'").sintoma, 
           y=phq_gf.query("variable=='Nunca'").value),
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()

In [54]:
fig=[go.Figure(data=[go.Bar(name=i, x=phq_gf[phq_gf['variable']==i].sintoma, y=phq_gf[phq_gf['variable']==i].value)]) for i in valores]

In [55]:
fig

[Figure({
     'data': [{'name': 'Nunca',
               'type': 'bar',
               'x': array(['s01', 's02', 's03', 's04', 's05', 's06', 's07', 's08', 's09'],
                          dtype=object),
               'y': array([44.01, 44.81, 46.05, 30.51, 50.66, 62.6 , 71.95, 72.68, 85.91])}],
     'layout': {'template': '...'}
 }),
 Figure({
     'data': [{'name': 'Algunos',
               'type': 'bar',
               'x': array(['s01', 's02', 's03', 's04', 's05', 's06', 's07', 's08', 's09'],
                          dtype=object),
               'y': array([40.93, 44.94, 34.59, 48.41, 33.1 , 25.59, 20.91, 19.25, 10.46])}],
     'layout': {'template': '...'}
 }),
 Figure({
     'data': [{'name': 'Todos',
               'type': 'bar',
               'x': array(['s01', 's02', 's03', 's04', 's05', 's06', 's07', 's08', 's09'],
                          dtype=object),
               'y': array([ 9.63,  5.68, 13.37, 13.5 , 10.77,  7.17,  4.26,  5.02,  2.04])}],
     'layout': {'templat

NameError: name 'df_sintom' is not defined

### Algoritmo K-modes

In [ ]:
#Algortimo con inicización de Cao
km_cao = KModes(n_clusters=4,
                init = "Cao",
                n_init = 1, verbose=1)
fitClusters_cao = km_cao.fit_predict(phq_rec)

In [ ]:
#Predicciones
fitClusters_cao

In [ ]:
#Moda de clusters por variable
cluster_cao = pd.DataFrame(km_cao.cluster_centroids_)
cluster_cao.columns = phq_ft.columns
cluster_cao

In [ ]:
#Algoritmo con inicialización de Huang
km_huang = KModes(n_clusters=4, init = "Huang", n_init = 1, verbose=1)
fitClusters_huang = km_huang.fit_predict(phq_rec)

In [ ]:
fitClusters_huang

In [ ]:
#Moda de clusters por variable
cluster_huang = pd.DataFrame(km_huang.cluster_centroids_)
cluster_huang.columns = phq_ft.columns
cluster_huang

## Número óptimo de clusters

In [ ]:
cost = []
for num_clusters in list(range(1,10)):
    kmode = KModes(n_clusters=num_clusters, init = "Cao", n_init = 1, verbose=0)
    kmode.fit_predict(phq_rec)
    cost.append(kmode.cost_)

In [ ]:
y = np.array([i for i in range(1,10,1)])
plt.plot(y,cost)

In [ ]:
### Se eligen 3
km_cao = KModes(n_clusters=3,
               init='Cao',
               n_init=1,
               verbose=0)
fit_kmcao=km_cao.fit_predict(phq_rec)

In [ ]:
phq_rec['cluster']= fit_kmcao

In [ ]:
phq_rec.head()

In [ ]:
#Moda de clusters por variable
cluster_cao3 = pd.DataFrame(km_cao.cluster_centroids_)
cluster_cao3.columns = phq_ft.drop('cluster',axis=1).columns
cluster_cao3